In [1]:
##Sentiment Analysis for Amazon Product Reviews


In [ ]:
import nltk
#nltk.download()

In [37]:
import pandas as pd
import numpy as np


In [38]:
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from bs4 import BeautifulSoup

In [39]:
wordnetlemmatizer = WordNetLemmatizer()

In [40]:
stopwords = set(w.rstrip() for w in open('stopwords.txt'))

In [41]:
#Load the Rewviews
positive_reviews = BeautifulSoup(open('./electronics/positive.review.txt').read())
positive_reviews = positive_reviews.findAll('review_text')

negative_reviews = BeautifulSoup(open('./electronics/negative.review.txt').read())
negative_reviews = negative_reviews.findAll('review_text')


C:\Program Files\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Program Files\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [42]:
#Shuffling randomly to have balanced classes
np.random.shuffle(positive_reviews)
positive_reviews = positive_reviews[:len(negative_reviews)]

### Tokenizing

In [44]:
#example
t = positive_reviews[0]
#print(t)
#nltk.tokenize.word_tokenize(t.text)

In [45]:
def my_tokenizer(s):
    s=s.lower()
    tokens = nltk.tokenize.word_tokenize(s) #Split string into words
    tokens = [t for t in tokens if len(t)>2] #remove short words
    tokens = [wordnetlemmatizer.lemmatize(t) for t in tokens] #put words into base form
    tokens = [t for t in tokens if t not in stopwords] #remove stopwords
    return tokens


In [46]:
#Create a word to index map to create word frequency vectors later
#Also save the tokenized versions
word_index_map = {}
current_index=0
positive_tokenized=[]
negative_tokenized=[]


In [47]:
for review in positive_reviews:
    tokens = my_tokenizer(review.text)
    positive_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index+=1

for review in negative_reviews:
    tokens = my_tokenizer(review.text)
    negative_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index+=1

    

In [51]:
#Create Input metrices
def tokens_to_vector(tokens,label):
    x = np.zeros(len(word_index_map)+1) #last element is for label
    for t in tokens:
        i=word_index_map[t]
        x[i]+=1
    x = x/x.sum() #normalize it before setting label
    x[-1]=label
    return x


In [52]:
N = len(positive_tokenized)+len(negative_tokenized)
#N * D+1 matrix: keeping them together now to shuffle later
data = np.zeros((N, len(word_index_map)+1))
i=0

for tokens in positive_tokenized:
    xy=tokens_to_vector(tokens,1)
    data[i,:]=xy
    i+=1

for tokens in negative_tokenized:
    xy=tokens_to_vector(tokens,0)
    data[i,:]=xy
    i+=1


In [54]:
data.shape

(2000, 11089)

In [56]:
#Shuffle the data to create test and train split
#do it multiple times
np.random.shuffle(data)
np.random.shuffle(data)
np.random.shuffle(data)
np.random.shuffle(data)


In [57]:
X = data[:,:-1]
Y = data[:,-1]

In [62]:
#Train and test split - last 100 rows test
XTrain = X[:-100,]
YTrain = Y[:-100,]
XTest = X[-100:,]
YTest = Y[-100:,]

In [66]:
YTest.shape


(100,)

### Build the model

In [67]:
model = LogisticRegression()
model.fit(XTrain,YTrain)
score = model.score(XTest,YTest)

In [68]:
score

0.77000000000000002

In [73]:
## Weights for each words:
threshold = 0.5
for word, index in word_index_map.iteritems():
    weight = model.coef_[0][index]
    if weight > threshold or weight<-threshold:
        print (word,weight)

ValueError: too many values to unpack (expected 2)